# 1. 전처리

In [1]:
# ngram 생성하기

import os
import re
from ekonlpy.sentiment import MPCK
import pandas as pd

TXT_DIR = "../../01_Crawling/financecrawler/TXT"
PDF_DIR = "../../01_Crawling/financecrawler/PDF"
data = pd.read_csv('../../01_Crawling/financecrawler/new.csv')
df = pd.DataFrame(columns=['date', 'content', 'scores', 'ngrams'])

text_list = os.listdir(TXT_DIR)
pdf_list = os.listdir(PDF_DIR)

ngram_list = []
scores = []
content = []
dates = []

for i in range(5):
    # 텍스트 불러오기
    text_file = open(TXT_DIR + f"/{text_list[i]}", 'r', encoding='utf-8-sig')
    text = ''
    for line in text_file:
        new_line = line.strip()
        if len(new_line) == 0:
            continue
        else:
            text += new_line

    re_text = re.sub('[^ ㄱ-ㅣ가-힣]+', '', text).lstrip().rstrip()
    
    content.append(re_text)
    dates.append(data['date'].loc[i])

    # MPCK 객체 생성
    mpck = MPCK()

    tokens = mpck.tokenize(re_text)
    ngrams = mpck.ngramize(tokens)
    score = mpck.classify(tokens+ngrams, intensity_cutoff=1.5)

    scores.append(score)
    ngram_list.append(ngrams)

In [2]:
dates

['11.01.03', '11.01.03', '11.01.03', '11.01.04', '11.01.05']

In [3]:
df['date'] = dates
df['date'] = pd.to_datetime(df['date'], yearfirst=True)
df['content'] = content
df['scores'] = scores
df['ngrams'] = ngram_list

In [ ]:
df.to_csv('bondreport.csv', mode='w',encoding='utf-8-sig')

# 2. 레이블링

In [8]:
import pandas as pd

bond_report = pd.read_csv('./bondreport.csv', encoding='utf-8-sig')
bond_report.head()

,Unnamed: 0,date,content,scores,ngrams
0,0,2011-01-03,본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보를 바탕으로 작성한 ...,"{'Polarity': -0.9999999863254809, 'Intensity':...","['국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG', '국고채/NNG;입찰..."
1,1,2011-01-03,본드브리프최종 을 위한 경기금리 바닥론과 순환성 이슈 저점대비 선 차 기술적 지지...,"{'Polarity': -1.0, 'Intensity': 1000000.0, 'Po...","['개선/NNG;경기/NNG;선행/NNG;지수/NNG;상승/NNG', '장단기스프레..."
2,2,2011-01-03,추가적인 금리상승이 불가피한 상황 금주 채권금리는 펀더멘탈 개선 수급부담 정책 ...,"{'Polarity': -0.9999999999098234, 'Intensity':...","['채권/NNG;금리/NNG;펀더멘털/NNG;개선/NNG', '출구전략/NNG;우려..."
3,3,2011-01-04,본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보를 바탕으로 작성한 ...,"{'Polarity': -0.9999973838141546, 'Intensity':...","['국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG', '금리/NNG;상승/..."
4,4,2011-01-05,본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보를 바탕으로 작성한 ...,"{'Polarity': -0.9997514848202238, 'Intensity':...","['금리/NNG;인상/NNG;가능성/NNG;경계/NNG;높/VA', '국고채/NNG..."


In [9]:
base_rate_label = pd.read_csv('../minutes_preprocessing/base_rate_label.csv', encoding='utf-8-sig')
base_rate_label.head()  

,date,rate,label
0,2011-01-01,-0.25,-1
1,2011-01-02,-0.25,-1
2,2011-01-03,-0.25,-1
3,2011-01-04,-0.25,-1
4,2011-01-05,-0.25,-1


In [10]:
bond_report.drop(['content', 'scores'], axis=1, inplace=True)
bond_report

,Unnamed: 0,date,ngrams
0,0,2011-01-03,"['국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG', '국고채/NNG;입찰..."
1,1,2011-01-03,"['개선/NNG;경기/NNG;선행/NNG;지수/NNG;상승/NNG', '장단기스프레..."
2,2,2011-01-03,"['채권/NNG;금리/NNG;펀더멘털/NNG;개선/NNG', '출구전략/NNG;우려..."
3,3,2011-01-04,"['국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG', '금리/NNG;상승/..."
4,4,2011-01-05,"['금리/NNG;인상/NNG;가능성/NNG;경계/NNG;높/VA', '국고채/NNG..."
...,...,...,...
3190,3190,2021-12-16,"['실업률/NNG;하락/NNG;임금/NNG;상승/NNG', '장단기스프레드/NNG;..."
3191,3191,2021-12-17,"['가격/NNG;상승/NNG;압력/NNG', '경제/NNG;불확실성/NNG;높/VA..."
3192,3192,2021-12-22,"['전망/NNG;상향/NNG;전망/NNG;하향/NNG', '금리/NNG;하락/NNG..."
3193,3193,2021-12-23,"['우려/NNG;완화/NNG;주가/NNG;상승/NNG', '채권/NNG;금리/NNG..."


In [11]:
bond_report

,Unnamed: 0,date,ngrams
0,0,2011-01-03,"['국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG', '국고채/NNG;입찰..."
1,1,2011-01-03,"['개선/NNG;경기/NNG;선행/NNG;지수/NNG;상승/NNG', '장단기스프레..."
2,2,2011-01-03,"['채권/NNG;금리/NNG;펀더멘털/NNG;개선/NNG', '출구전략/NNG;우려..."
3,3,2011-01-04,"['국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG', '금리/NNG;상승/..."
4,4,2011-01-05,"['금리/NNG;인상/NNG;가능성/NNG;경계/NNG;높/VA', '국고채/NNG..."
...,...,...,...
3190,3190,2021-12-16,"['실업률/NNG;하락/NNG;임금/NNG;상승/NNG', '장단기스프레드/NNG;..."
3191,3191,2021-12-17,"['가격/NNG;상승/NNG;압력/NNG', '경제/NNG;불확실성/NNG;높/VA..."
3192,3192,2021-12-22,"['전망/NNG;상향/NNG;전망/NNG;하향/NNG', '금리/NNG;하락/NNG..."
3193,3193,2021-12-23,"['우려/NNG;완화/NNG;주가/NNG;상승/NNG', '채권/NNG;금리/NNG..."


In [12]:
merge_df = pd.merge(left=bond_report, right=base_rate_label, how='left', on='date')

In [13]:
merge_df.drop(['rate'], axis=1, inplace=True)

In [14]:
# merge_df.to_csv('merge_bond_rate_label.csv', encoding='utf-8-sig')

In [26]:
# str to list

for i in range(len(merge_df)):
    ng = merge_df['ngrams'][i].replace("'", '')
    # print(ng)
    # print()
    ng2 = ng.replace(ng[0],'').replace(ng[-1], '').replace(' ', '')
    # print(ng2)
    # print()
    ng3 = ng2.split(',')
    merge_df.at[i, 'ngrams'] = ng3

# 3. 모델링
- Market   
NBC 조건부 확률 계산 시 ngram을 카운팅하게 될텐데 nltk 라이브러리 사용하면 빠르다.
bagging 하여 polarity score 계산, gray area 제외하여 hawkish, dovish 사전 구축

In [51]:
import nltk
import itertools

ngram_list = list(itertools.chain(*(list(merge_df['ngrams']))))
up_ngram_list = list(itertools.chain(*(list(merge_df[merge_df['label']==1]['ngrams']))))
down_ngram_list = list(itertools.chain(*(list(merge_df[merge_df['label']==-1]['ngrams']))))

In [52]:
fdist = nltk.FreqDist(ngram_list) 
count_df = pd.DataFrame(list(zip(fdist.keys(), fdist.values())))
count_df.columns = ['ngrams', 'n_sums']

up_fdist = nltk.FreqDist(up_ngram_list)
up_count_df = pd.DataFrame(list(zip(up_fdist.keys(), up_fdist.values())))
up_count_df.columns = ['ngrams', 'n_sums']

down_fdist = nltk.FreqDist(down_ngram_list)
down_count_df = pd.DataFrame(list(zip(down_fdist.keys(), down_fdist.values())))
down_count_df.columns = ['ngrams', 'n_sums']

In [53]:
count_df.columns = ['ngrams', 'n_sums']
count_df

,ngrams,n_sums
0,국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG,10
1,국고채/NNG;입찰/NNG;물량/NNG;부담/NNG,4
2,금리/NNG;상승/NNG;압력/NNG;높/VA,23
3,장기물/NNG;순매수/NNG;강화/NNG,12
4,예상/NNG;채권시장/NNG;강세/NNG,11
...,...,...
26336,우려/NNG;완화/NNG;주가/NNG;상승/NNG,1
26337,국고채/NNG;금리/NNG;전고점/NNG,1
26338,인상/NNG;경로/NNG;불확실성/NNG,1
26339,완화/NNG;주식시장/NNG;강세/NNG,1


In [54]:
ngrams_df = pd.merge(left = count_df, right = up_count_df, how='left', on='ngrams')
ngrams_df # x=count_df, y=up_count_df

,ngrams,n_sums_x,n_sums_y
0,국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG,10,NaN
1,국고채/NNG;입찰/NNG;물량/NNG;부담/NNG,4,NaN
2,금리/NNG;상승/NNG;압력/NNG;높/VA,23,1.0
3,장기물/NNG;순매수/NNG;강화/NNG,12,1.0
4,예상/NNG;채권시장/NNG;강세/NNG,11,NaN
...,...,...,...
26336,우려/NNG;완화/NNG;주가/NNG;상승/NNG,1,NaN
26337,국고채/NNG;금리/NNG;전고점/NNG,1,NaN
26338,인상/NNG;경로/NNG;불확실성/NNG,1,NaN
26339,완화/NNG;주식시장/NNG;강세/NNG,1,NaN


In [56]:
ngrams_df = pd.merge(left = ngrams_df, right = down_count_df, how='left', on='ngrams') # n_sums=down_count_df

In [57]:
ngrams_df.columns = ['ngrams', 'n_count_df', 'n_up_count_df', 'n_down_count_df']

In [59]:
ngrams_df.fillna(0)

,ngrams,n_count_df,n_up_count_df,n_down_count_df
0,국고채/NNG;스프레드/NNG;축소/NNG;압력/NNG,10,0.0,8.0
1,국고채/NNG;입찰/NNG;물량/NNG;부담/NNG,4,0.0,3.0
2,금리/NNG;상승/NNG;압력/NNG;높/VA,23,1.0,4.0
3,장기물/NNG;순매수/NNG;강화/NNG,12,1.0,8.0
4,예상/NNG;채권시장/NNG;강세/NNG,11,0.0,5.0
...,...,...,...,...
26336,우려/NNG;완화/NNG;주가/NNG;상승/NNG,1,0.0,1.0
26337,국고채/NNG;금리/NNG;전고점/NNG,1,0.0,1.0
26338,인상/NNG;경로/NNG;불확실성/NNG,1,0.0,1.0
26339,완화/NNG;주식시장/NNG;강세/NNG,1,0.0,1.0
